In [48]:
import collections
from ortools.sat.python import cp_model

In [65]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""
    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def on_solution_callback(self):
        """Called at each new solution."""
        print('Solution %i, time = %f s, objective = %i' % (self.__solution_count, self.WallTime(), self.ObjectiveValue()))
        self.__solution_count += 1

In [18]:
data = {
    "supply_sites": ["1", "2", "3", "4"],
    "demand_sites": ["a", "b", "c", "d", "e"],
    "demand_data": {
        "a": {
            "task0": [{"processing_time":  4, "machine_id":  6, "due_date":  9}],
            "task1": [{"processing_time":  4, "machine_id":  6, "due_date":  9}]
        },

    }
}

In [23]:
from pprint import pprint

dict_ = data["demand_data"]

for index, (key, value) in enumerate(dict_.items()):
    for i, (k, v) in enumerate(dict_[key].items()):
        for i2, k2 in enumerate(dict_[key][k]):
            print(i2, k2)

0 {'processing_time': 4, 'machine_id': 6, 'due_date': 9}
0 {'processing_time': 4, 'machine_id': 6, 'due_date': 9}


In [4]:
"""Solve a small flexible jobshop problem."""

# Model the flexible jobshop problem.
model = cp_model.CpModel()

# Data part.
jobs = [  # task = (processing_time, machine_id, due_date)
    [  # Job 0
        [(3, 0, 40), (1, 1, 40), (5, 2, 40)],  # task 0 with 3 alternatives
        [(2, 0, 40), (4, 1, 40), (6, 2, 40)],  # task 1 with 3 alternatives
        [(2, 0, 5), (3, 1, 40), (1, 2, 40)],  # task 2 with 3 alternatives
        # [(2, 0, 5), (3, 1, 40)],  # task 2 with 3 alternatives
    ],
    [  # Job 1
        [(2, 0, 40), (3, 1, 40), (4, 2, 40)],
        [(1, 0, 40), (5, 1, 40), (4, 2, 40)],
        [(2, 0, 40), (1, 1, 40), (4, 2, 40)],
    ],
    [  # Job 2
        [(2, 0, 40), (1, 1, 40), (4, 2, 40)],
        [(2, 0, 40), (3, 1, 40), (4, 2, 40)],
        [(3, 0, 40), (1, 1, 40), (5, 2, 40)],
    ],
]


num_jobs = len(jobs)
all_jobs = range(num_jobs)

num_machines = 3
all_machines = range(num_machines)



horizon = 0
for job in jobs:
    for task in job:
        max_task_duration = 0
        for alternative in task:
            max_task_duration = max(max_task_duration, alternative[0])
        horizon += max_task_duration

print('Horizon = %i' % horizon)

NameError: name 'cp_model' is not defined

In [67]:
# Global storage of variables.
intervals_per_resources = collections.defaultdict(list)
starts = {}  # indexed by (job_id, task_id).
ends_ = {}  # indexed by (job_id, task_id).
presences = {}  # indexed by (job_id, task_id, alt_id).
job_ends = []
all_tasks = {}
# Named tuple to store information about created variables.
task_type = collections.namedtuple('task_type', 'start end deadline interval')


In [68]:
# Scan the jobs and create the relevant variables and intervals.
for job_id in all_jobs:
    job = jobs[job_id]
    num_tasks = len(job)
    previous_end = None
    for task_id in range(num_tasks):
        task = job[task_id]

        min_duration = task[0][0]
        max_duration = task[0][0]

        num_alternatives = len(task)
        all_alternatives = range(num_alternatives)

        for alt_id in range(1, num_alternatives):
            alt_duration = task[alt_id][0]
            min_duration = min(min_duration, alt_duration)
            max_duration = max(max_duration, alt_duration)
        
        deadline = task[0][2]

        # Create main interval for the task.
        suffix_name = '_j%i_t%i' % (job_id, task_id)
        start = model.NewIntVar(0, horizon, 'start' + suffix_name)
        duration = model.NewIntVar(min_duration, max_duration,'duration' + suffix_name)
        end = model.NewIntVar(0, horizon, 'end' + suffix_name)
        interval = model.NewIntervalVar(start, duration, end, 'interval' + suffix_name)

        deadline_var = model.NewIntVar(deadline, deadline,'deadline' + suffix_name)
        all_tasks[job_id, task_id] = task_type(start=start, end=end, deadline=deadline_var, interval=interval)


        # Store the start for the solution.
        starts[(job_id, task_id)] = start

        # Store the end for the solution. ## Metin
        ends_[(job_id, task_id)] = end

        # Add precedence with previous task in the same job.
        if previous_end is not None:
            model.Add(start >= previous_end)
        previous_end = end

        # Create alternative intervals.
        if num_alternatives > 1:
            l_presences = []
            for alt_id in all_alternatives:
                alt_suffix = '_j%i_t%i_a%i' % (job_id, task_id, alt_id)
                l_presence = model.NewBoolVar('presence' + alt_suffix)
                l_start = model.NewIntVar(0, horizon, 'start' + alt_suffix)
                l_duration = task[alt_id][0]
                l_end = model.NewIntVar(0, horizon, 'end' + alt_suffix)
                l_interval = model.NewOptionalIntervalVar(l_start, l_duration, l_end, l_presence, 'interval' + alt_suffix)
                l_presences.append(l_presence)

                # Link the master variables with the local ones.
                model.Add(start == l_start).OnlyEnforceIf(l_presence)
                model.Add(duration == l_duration).OnlyEnforceIf(l_presence)
                model.Add(end == l_end).OnlyEnforceIf(l_presence)

                # Add the local interval to the right machine.
                intervals_per_resources[task[alt_id][1]].append(l_interval)

                # Store the presences for the solution.
                presences[(job_id, task_id, alt_id)] = l_presence

            # Select exactly one presence variable.
            model.AddExactlyOne(l_presences)
        else:
            intervals_per_resources[task[0][1]].append(interval)
            presences[(job_id, task_id, 0)] = model.NewConstant(1)

    job_ends.append(previous_end)

In [69]:
# deadline
for job_id, job in enumerate(jobs):
    for task_id, task in enumerate(job):
        model.Add(all_tasks[job_id, task_id].end <= all_tasks[job_id, task_id].deadline)

In [70]:
# Create machines constraints.
for machine_id in all_machines:
    intervals = intervals_per_resources[machine_id]
    if len(intervals) > 1:
        model.AddNoOverlap(intervals)

In [71]:
# Makespan objective
makespan = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(makespan, job_ends)
model.Minimize(makespan)

# Solve model.
solver = cp_model.CpSolver()
solution_printer = SolutionPrinter()
status = solver.Solve(model, solution_printer)

Solution 0, time = 0.012499 s, objective = 9
Solution 1, time = 0.013771 s, objective = 8
Solution 2, time = 0.013956 s, objective = 7


In [72]:
# Print final solution.
for job_id in all_jobs:
    print('Job %i:' % job_id)
    for task_id in range(len(jobs[job_id])):
        start_value = solver.Value(starts[(job_id, task_id)])
        machine = -1
        duration = -1
        selected = -1
        for alt_id in range(len(jobs[job_id][task_id])):
            if solver.Value(presences[(job_id, task_id, alt_id)]):
                duration = jobs[job_id][task_id][alt_id][0]
                machine = jobs[job_id][task_id][alt_id][1]
                selected = alt_id
        print(
            '  task_%i_%i starts at %i (alt %i, machine %i, duration %i)' %
            (job_id, task_id, start_value, selected, machine, duration))

print('Solve status: %s' % solver.StatusName(status))
print('Optimal objective value: %i' % solver.ObjectiveValue())
print('Statistics')
print('  - conflicts : %i' % solver.NumConflicts())
print('  - branches  : %i' % solver.NumBranches())
print('  - wall time : %f s' % solver.WallTime())

Job 0:
  task_0_0 starts at 0 (alt 1, machine 1, duration 1)
  task_0_1 starts at 1 (alt 0, machine 0, duration 2)
  task_0_2 starts at 3 (alt 0, machine 0, duration 2)
Job 1:
  task_1_0 starts at 0 (alt 2, machine 2, duration 4)
  task_1_1 starts at 5 (alt 0, machine 0, duration 1)
  task_1_2 starts at 6 (alt 1, machine 1, duration 1)
Job 2:
  task_2_0 starts at 1 (alt 1, machine 1, duration 1)
  task_2_1 starts at 2 (alt 1, machine 1, duration 3)
  task_2_2 starts at 5 (alt 1, machine 1, duration 1)
Solve status: OPTIMAL
Optimal objective value: 7
Statistics
  - conflicts : 0
  - branches  : 72
  - wall time : 0.018438 s


In [ ]:
"""
Job 0:
  task_0_0 starts at 2 (alt 1, machine 1, duration 1)
  task_0_1 starts at 3 (alt 0, machine 0, duration 2)
  task_0_2 starts at 5 (alt 2, machine 2, duration 1)
Job 1:
  task_1_0 starts at 0 (alt 0, machine 0, duration 2)
  task_1_1 starts at 2 (alt 0, machine 0, duration 1)
  task_1_2 starts at 4 (alt 1, machine 1, duration 1)
Job 2:
  task_2_0 starts at 0 (alt 1, machine 1, duration 1)
  task_2_1 starts at 1 (alt 2, machine 2, duration 4)
  task_2_2 starts at 5 (alt 1, machine 1, duration 1)
Solve status: OPTIMAL
Optimal objective value: 6
Statistics
  - conflicts : 0
  - branches  : 108
  - wall time : 0.017853 s
"""